In [1]:
!unzip data.zip

Archive:  data.zip
  inflating: train.csv               
  inflating: val.csv                 


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 38.3 MB/s 
     |████████████████████████████████| 120 kB 68.4 MB/s 
     |████████████████████████████████| 6.6 MB 53.0 MB/s 


In [3]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from tqdm import tqdm
from collections import Counter
from transformers import pipeline
import pandas as pd

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
sentiment = pipeline('text-classification', model='sismetanin/rubert-toxic-pikabu-2ch')

Downloading:   0%|          | 0.00/920 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/712M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/540 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [6]:
sentiment('это унылое...')

[{'label': 'LABEL_0', 'score': 0.9707620143890381}]

In [7]:
df_train = pd.read_csv('train.csv')
df_val = pd.read_csv('val.csv')

In [8]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [9]:
df_train['class'].value_counts()

1    92063
0    89404
Name: class, dtype: int64

In [10]:
idx = 1
print('Example:', df_train.iloc[idx][1])
print('True_label:', df_train.iloc[idx][2])
print('Model predict:', sentiment(df_train.iloc[idx][1])[0])

Example: RT @GalyginVadim: Ребята и девчата!
Все в кино!!! "Вот Это Любовь!"
Сегодня! Завтра!  И потом!)))))
#вотэтолюбовь
True_label: 1
Model predict: {'label': 'LABEL_1', 'score': 0.6654629111289978}


In [11]:
df_train['text'] = df_train['text'].apply(lambda x: x.lower())
df_val['text'] = df_val['text'].apply(lambda x: x.lower())

In [12]:
from transformers import BertTokenizer

In [13]:
tokenizer = BertTokenizer.from_pretrained('sismetanin/rubert-toxic-pikabu-2ch')

In [14]:
max_length = 30
test = df_train.iloc[idx][1]

In [15]:
test_token = tokenizer(test,padding='max_length', max_length=max_length, 
                       truncation=True, return_tensors="pt"  )

In [16]:
test_token.get('input_ids')

tensor([[  101,   266,   271,   168, 79844, 49482,  3834, 10874, 11215,   257,
           156, 47509,   851, 85353,  4292,   106,  4752,   845,  7551,   106,
           106,   106,   108, 19030,  3998, 21361,   106,   108, 12904,   102]])

In [17]:
print({x : tokenizer.encode(x) for x in test[:max_length].split()})

{'rt': [101, 266, 271, 102], '@galyginvadim:': [101, 168, 79844, 49482, 3834, 10874, 11215, 257, 156, 102], 'ребята': [101, 47509, 102], 'и': [101, 851, 102], 'дев': [101, 7631, 102]}


In [18]:
test = tokenizer.decode(test_token.input_ids[0])

print(test)

[CLS] rt @ galyginvadim : ребята и девчата! все в кино!!! " вот это любовь! " сегодня [SEP]


In [19]:
class TwitterDataset(torch.utils.data.Dataset):
    
    def __init__(self, txts, labels):
        self._labels = labels
        
        self.tokenizer = BertTokenizer.from_pretrained('sismetanin/rubert-toxic-pikabu-2ch')
        self._txts = [self.tokenizer(text, padding='max_length', max_length=10,
                                     truncation=True, return_tensors="pt")
                      for text in txts]
        
    def __len__(self):
        return len(self._txts)
    
    def __getitem__(self, index):
        return self._txts[index], self._labels[index]

In [20]:
y_train = df_train['class'].values
y_val = df_val['class'].values

train_dataset = TwitterDataset(df_train['text'], y_train)
valid_dataset = TwitterDataset(df_val['text'], y_val)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=64,
                          shuffle=True,
                          num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=1)

In [21]:
for txt, lbl in train_loader:
    print(txt.keys())
    print(txt['input_ids'].shape)
    break

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
torch.Size([64, 1, 10])


In [22]:
from torch import nn
from transformers import BertModel


class BertClassifier(nn.Module):

    def __init__(self):
        super().__init__()
        self.bert = BertModel.from_pretrained('sismetanin/rubert-toxic-pikabu-2ch')
        # self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.sigm = nn.Sigmoid()

    def forward(self, x, mask):
        
        _, pooled_output = self.bert(input_ids=x, attention_mask=mask, return_dict=False)
        # _, pooled_output - набор эмбеддинигов слов, эмбеддинг предложения
        # dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(pooled_output)
        final_layer = self.sigm(linear_output)
        return final_layer

In [23]:
model = BertClassifier().to(device)
criterion = nn.CrossEntropyLoss()

# optimizer = Adam(model.parameters(), lr=0.001)  # полное обучение
optimizer = Adam(model.linear.parameters(), lr=0.001)  # неполное обучение

Some weights of the model checkpoint at sismetanin/rubert-toxic-pikabu-2ch were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
print(model)
print("Parameters full train:", sum([param.nelement() for param in model.parameters()]))
print("Parameters transfer learning:", sum([param.nelement() for param in model.linear.parameters()]))

BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [25]:
for epoch_num in range(2):
    total_acc_train = 0
    total_loss_train = 0

    model.train()
    for train_input, train_label in tqdm(train_loader):
        mask = train_input['attention_mask'].to(device)
        input_id = train_input['input_ids'].squeeze(1).to(device)
        train_label = train_label.to(device)

        output = model(input_id, mask)
                
        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()
                
        acc = (output.argmax(dim=1) == train_label).sum().item()
        total_acc_train += acc

        model.zero_grad()
        batch_loss.backward()
        optimizer.step()
            
    model.eval()
    total_loss_val, total_acc_val = 0.0, 0.0
    for val_input, val_label in valid_loader:
        val_label = val_label.to(device)
        mask = val_input['attention_mask'].to(device)
        input_id = val_input['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask)

        batch_loss = criterion(output, val_label)
        total_loss_val += batch_loss.item()
                    
        acc = (output.argmax(dim=1) == val_label).sum().item()
        total_acc_val += acc
            
    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train Accuracy: {total_acc_train / len(train_dataset): .3f} \
        | Val Loss: {total_loss_val / len(valid_dataset): .3f} \
        | Val Accuracy: {total_acc_val / len(valid_dataset): .3f}')

100%|██████████| 2836/2836 [05:15<00:00,  8.99it/s]


Epochs: 1 | Train Loss:  0.010         | Train Accuracy:  0.589         | Val Loss:  0.010         | Val Accuracy:  0.604


100%|██████████| 2836/2836 [05:14<00:00,  9.01it/s]


Epochs: 2 | Train Loss:  0.010         | Train Accuracy:  0.601         | Val Loss:  0.010         | Val Accuracy:  0.607
